In [ ]:
# pip show earthengine-api geemap ipywidgets ipython

# Version 1
- Flood Frequency 
- Flood Pronearea
- Slope Flat area from SRTM
- Montly water surface layer by Time period 

In [ ]:
# import ee
# import geemap
# import datetime
# import ipywidgets as widgets
# from IPython.display import display

# # Initialize Earth Engine API
# ee.Initialize()

# # Create map
# Map = geemap.Map()

# # Define region of interest (ROI) for Guyana
# gaul_level1 = ee.FeatureCollection('FAO/GAUL/2015/level1')
# guyana = gaul_level1.filter(ee.Filter.eq('ADM0_NAME', 'Guyana'))
# georgetown = guyana.filter(ee.Filter.eq('ADM1_NAME', 'Georgetown'))
# roi = guyana.geometry()

# # Define dynamic time period
# start_date = datetime.date(1984, 3, 16)
# end_date = datetime.date(2024, 12, 31)

# # Convert to ee.Date objects
# start_date_ee = ee.Date.fromYMD(start_date.year, start_date.month, start_date.day)
# end_date_ee = ee.Date.fromYMD(end_date.year, end_date.month, end_date.day)

# # Load datasets
# jrc = ee.ImageCollection("JRC/GSW1_4/MonthlyHistory")
# srtm = ee.Image("USGS/SRTMGL1_003")

# # Filter JRC dataset for the selected time period and region
# jrc_filtered = jrc.filterBounds(roi).filterDate(start_date_ee, end_date_ee)

# # Step 1: Exclude permanent water bodies
# permanent_water = ee.Image("JRC/GSW1_4/GlobalSurfaceWater").select("occurrence").gte(90)
# jrc_filtered = jrc_filtered.map(lambda img: img.updateMask(permanent_water.Not()))

# # Step 2: Calculate Flood Frequency
# def calculate_flood_frequency(collection):
#     # Detect observations and water occurrences
#     def add_bands(img):
#         obs = img.gt(0).rename("obs")
#         water = img.select("water").eq(2).rename("water")
#         return img.addBands([obs, water])
    
#     collection = collection.map(add_bands)
#     total_obs = collection.select("obs").sum().rename("total_obs")
#     total_water = collection.select("water").sum().rename("total_water")
#     flood_frequency = total_water.divide(total_obs).multiply(100).rename("flood_frequency")
#     return flood_frequency.updateMask(flood_frequency.neq(0))

# flood_frequency = calculate_flood_frequency(jrc_filtered)

# # Step 3: Analyze DEM
# # Calculate slope and flat areas
# slope = ee.Terrain.slope(srtm).rename("slope")
# flat_area = slope.lt(5).rename("flat_area")  
# low_lying = srtm.lt(10).rename("low_lying")  

# # Step 4: Flood-Prone Areas
# flood_prone_area = flood_frequency.updateMask(low_lying.And(flat_area))

# # Step 5: Monthly Water Surface Maps
# def get_monthly_flood_layers(collection, roi, permanent_water):
#     """
#     Extracts monthly flood layers from the given collection,
#     excluding permanent water bodies.
#     """
#     monthly_layers = {}
#     months = collection.aggregate_array("system:index").getInfo()
    
#     for month in months:
#         img = collection.filter(ee.Filter.eq("system:index", month)).first()
#         water_layer = img.select("water").eq(2).updateMask(permanent_water.Not())
#         monthly_layers[month] = water_layer.clip(roi)
    
#     return monthly_layers

# # Call the function with the permanent water exclusion
# monthly_layers = get_monthly_flood_layers(jrc_filtered, roi, permanent_water)

# # Visualization parameters
# dem_vis = {"min": 0, "max": 50, "palette": ["ffffff", "00ff00", "007f00"]}
# slope_vis = {"min": 0, "max": 30, "palette": ["ffffff", "ffcc99", "ff3300"]}
# flat_area_vis = {"min": 0, "max": 1, "palette": ["ffffff", "0000ff"]}
# flood_frequency_vis = {"min": 0, "max": 50, "palette": ["ffffff", "fffcb8", "0905ff"]}
# flood_prone_area_vis = {"min": 0, "max": 50, "palette": ["ffffff", "ff9999", "ff0000"]}
# water_surface_vis = {"min": 0, "max": 1, "palette": ["0000ff"]}

# # Interactive widgets
# start_date_picker = widgets.DatePicker(description="Start Date", value=start_date)
# end_date_picker = widgets.DatePicker(description="End Date", value=end_date)
# run_button = widgets.Button(description="Run Analysis", button_style="success")


# def update_map(change):
#     start_date = start_date_picker.value
#     end_date = end_date_picker.value

#     # Convert to ee.Date objects
#     start_date_ee = ee.Date.fromYMD(start_date.year, start_date.month, start_date.day)
#     end_date_ee = ee.Date.fromYMD(end_date.year, end_date.month, end_date.day)

#     # Re-filter the JRC dataset with the new date range
#     jrc_filtered = jrc.filterBounds(roi).filterDate(start_date_ee, end_date_ee)
#     # Exclude permanent water bodies
#     jrc_filtered = jrc_filtered.map(lambda img: img.updateMask(permanent_water.Not()))
#     # Calculate flood frequency
#     flood_frequency = calculate_flood_frequency(jrc_filtered)
#     # Calculate flood-prone areas
#     flood_prone_area = flood_frequency.updateMask(low_lying.And(flat_area))
#     # flood_prone_area = flood_frequency.updateMask(flat_area)
    
#     # Add new layers without removing existing ones
#     Map.centerObject(roi, 8)
#     Map.addLayer(srtm.clip(roi), dem_vis, "SRTM DEM")
#     Map.addLayer(slope.clip(roi), slope_vis, "Slope")
#     Map.addLayer(flood_frequency.clip(roi), flood_frequency_vis, f"Flood Frequency from {start_date} to {end_date}")
#     Map.addLayer(flood_prone_area.clip(roi), flood_prone_area_vis, f"Flood-Prone Areas from {start_date} to {end_date}")
   
#     # for month, layer in monthly_layers.items():
#     #     Map.addLayer(layer, water_surface_vis, f"Water Surface {month}")

# # Set the button action
# run_button.on_click(update_map)

# # Display the widgets
# display(start_date_picker, end_date_picker, run_button)

# # Show the map
# Map.addLayerControl()
# Map


# Version 2 
* added low-lying buttom

In [ ]:
import ee
import geemap
import datetime
import ipywidgets as widgets
from IPython.display import display

# Initialize Earth Engine API
ee.Initialize()

# Create map
Map = geemap.Map()

# Define region of interest (ROI) for Guyana
gaul_level1 = ee.FeatureCollection('FAO/GAUL/2015/level1')
guyana = gaul_level1.filter(ee.Filter.eq('ADM0_NAME', 'Guyana'))
georgetown = guyana.filter(ee.Filter.eq('ADM1_NAME', 'Georgetown'))
roi = guyana.geometry()

# Define dynamic time period
start_date = datetime.date(1984, 3, 16)
end_date = datetime.date(2024, 12, 31)

# Convert to ee.Date objects
start_date_ee = ee.Date.fromYMD(start_date.year, start_date.month, start_date.day)
end_date_ee = ee.Date.fromYMD(end_date.year, end_date.month, end_date.day)

# Load datasets
jrc = ee.ImageCollection("JRC/GSW1_4/MonthlyHistory")
srtm = ee.Image("USGS/SRTMGL1_003")

# Filter JRC dataset for the selected time period and region
jrc_filtered = jrc.filterBounds(roi).filterDate(start_date_ee, end_date_ee)

# Step 1: Exclude permanent water bodies
permanent_water = ee.Image("JRC/GSW1_4/GlobalSurfaceWater").select("occurrence").gte(90)
jrc_filtered = jrc_filtered.map(lambda img: img.updateMask(permanent_water.Not()))

# Step 2: Calculate Flood Frequency
def calculate_flood_frequency(collection):
    def add_bands(img):
        obs = img.gt(0).rename("obs")
        water = img.select("water").eq(2).rename("water")
        return img.addBands([obs, water])
    
    collection = collection.map(add_bands)
    total_obs = collection.select("obs").sum().rename("total_obs")
    total_water = collection.select("water").sum().rename("total_water")
    flood_frequency = total_water.divide(total_obs).multiply(100).rename("flood_frequency")
    return flood_frequency.updateMask(flood_frequency.neq(0))

flood_frequency = calculate_flood_frequency(jrc_filtered)

# Step 3: Analyze DEM
slope = ee.Terrain.slope(srtm).rename("slope")
flat_area = slope.lt(5).rename("flat_area")

# Default low-lying threshold (now changeable via UI)
low_lying_threshold = 10
low_lying = srtm.lt(low_lying_threshold).rename("low_lying")

# Step 4: Flood-Prone Areas
flood_prone_area = flood_frequency.updateMask(low_lying.And(flat_area))

# Visualization parameters
dem_vis = {"min": 0, "max": 50, "palette": ["ffffff", "00ff00", "007f00"]}
slope_vis = {"min": 0, "max": 30, "palette": ["ffffff", "ffcc99", "ff3300"]}
flat_area_vis = {"min": 0, "max": 1, "palette": ["ffffff", "0000ff"]}
flood_frequency_vis = {"min": 0, "max": 50, "palette": ["ffffff", "fffcb8", "0905ff"]}
flood_prone_area_vis = {"min": 0, "max": 50, "palette": ["ffffff", "ff9999", "ff0000"]}

# Interactive widgets
start_date_picker = widgets.DatePicker(description="Start Date", value=start_date)
end_date_picker = widgets.DatePicker(description="End Date", value=end_date)
low_lying_input = widgets.BoundedFloatText(description="Low-Lying Level (m)", value=low_lying_threshold, min=0, max=100, step=1)

# Dropdown for selecting base map
base_map_selector = widgets.Dropdown(
    options=["SATELLITE", "OSM"],
    value="SATELLITE",
    description="Base Map:"
)

run_button = widgets.Button(description="Run Analysis", button_style="success")

def update_map(change):
    start_date = start_date_picker.value
    end_date = end_date_picker.value
    low_lying_threshold = low_lying_input.value  # Get updated low-lying level
    selected_basemap = base_map_selector.value  # Get selected base map

    # Convert to ee.Date objects
    start_date_ee = ee.Date.fromYMD(start_date.year, start_date.month, start_date.day)
    end_date_ee = ee.Date.fromYMD(end_date.year, end_date.month, end_date.day)

    # Re-filter the JRC dataset with the new date range
    jrc_filtered = jrc.filterBounds(roi).filterDate(start_date_ee, end_date_ee)
    jrc_filtered = jrc_filtered.map(lambda img: img.updateMask(permanent_water.Not()))

    # Calculate flood frequency
    flood_frequency = calculate_flood_frequency(jrc_filtered)

    # Recalculate low-lying areas dynamically
    low_lying = srtm.lt(low_lying_threshold).rename("low_lying")

    # Calculate flood-prone areas
    flood_prone_area = flood_frequency.updateMask(low_lying.And(flat_area))

    # Clear only data layers (keeping the base map)
    Map.layers = Map.layers[:2]  
    
    # Set the selected base map
    Map.add_basemap(selected_basemap)

    # Keep base layers intact
    Map.centerObject(roi, 8)
    Map.addLayer(srtm.clip(roi), dem_vis, "SRTM DEM")
    Map.addLayer(slope.clip(roi), slope_vis, "Slope")
    Map.addLayer(flood_frequency.clip(roi), flood_frequency_vis, f"Flood Frequency ({start_date} to {end_date})")
    Map.addLayer(flood_prone_area.clip(roi), flood_prone_area_vis, f"Flood-Prone Areas ({start_date} to {end_date}) - Low-Lying < {low_lying_threshold}m")


# Set the button action
run_button.on_click(update_map)

# Display the widgets
display(start_date_picker, end_date_picker, low_lying_input, base_map_selector, run_button)

# Ensure the base map is added initially
Map.add_basemap("SATELLITE")  
Map.addLayerControl()
Map


In [ ]:
import ee
import geemap
import datetime
import ipywidgets as widgets
from IPython.display import display

# Initialize Earth Engine API
ee.Initialize()

# Create map
Map = geemap.Map()

# Load GAUL dataset and filter for Guyana
gaul_level1 = ee.FeatureCollection('FAO/GAUL/2015/level2')
guyana = gaul_level1.filter(ee.Filter.eq('ADM0_NAME', 'Guyana'))

# Get list of available cities in Guyana
city_list = guyana.aggregate_array('ADM1_NAME').getInfo()

# Define dynamic time period
start_date = datetime.date(1984, 3, 16)
end_date = datetime.date(2024, 12, 31)

# Convert to ee.Date objects
start_date_ee = ee.Date.fromYMD(start_date.year, start_date.month, start_date.day)
end_date_ee = ee.Date.fromYMD(end_date.year, end_date.month, end_date.day)

# Load datasets
jrc = ee.ImageCollection("JRC/GSW1_4/MonthlyHistory")
srtm = ee.Image("USGS/SRTMGL1_003")

# Interactive widgets
city_selector = widgets.Dropdown(
    options=city_list,
    value=city_list[0],
    description="City:",
)
start_date_picker = widgets.DatePicker(description="Start Date", value=start_date)
end_date_picker = widgets.DatePicker(description="End Date", value=end_date)
low_lying_input = widgets.BoundedFloatText(description="Low-Lying Level (m)", value=10, min=0, max=100, step=1)
base_map_selector = widgets.Dropdown(
    options=["SATELLITE", "OSM"],
    value="SATELLITE",
    description="Base Map:",
)
run_button = widgets.Button(description="Run Analysis", button_style="success")

    
def update_map(change):
    selected_city = city_selector.value
    roi = guyana.filter(ee.Filter.eq('ADM1_NAME', selected_city)).geometry()
    
    start_date = start_date_picker.value
    end_date = end_date_picker.value
    low_lying_threshold = low_lying_input.value  # Get updated low-lying level
    selected_basemap = base_map_selector.value  # Get selected base map
    
    # Convert to ee.Date objects
    start_date_ee = ee.Date.fromYMD(start_date.year, start_date.month, start_date.day)
    end_date_ee = ee.Date.fromYMD(end_date.year, end_date.month, end_date.day)
    
    # Filter datasets to ROI
    jrc_filtered = jrc.filterBounds(roi).filterDate(start_date_ee, end_date_ee)
    permanent_water = ee.Image("JRC/GSW1_4/GlobalSurfaceWater").select("occurrence").gte(90)
    jrc_filtered = jrc_filtered.map(lambda img: img.updateMask(permanent_water.Not()))
    
    # Calculate flood frequency
    def calculate_flood_frequency(collection):
        def add_bands(img):
            obs = img.gt(0).rename("obs")
            water = img.select("water").eq(2).rename("water")
            return img.addBands([obs, water])
        
        collection = collection.map(add_bands)
        total_obs = collection.select("obs").sum().rename("total_obs")
        total_water = collection.select("water").sum().rename("total_water")
        flood_frequency = total_water.divide(total_obs).multiply(100).rename("flood_frequency")
        return flood_frequency.updateMask(flood_frequency.neq(0))
    
    flood_frequency = calculate_flood_frequency(jrc_filtered)
    slope = ee.Terrain.slope(srtm).rename("slope")
    flat_area = slope.lt(5).rename("flat_area")
    low_lying = srtm.lt(low_lying_threshold).rename("low_lying")
    flood_prone_area = flood_frequency.updateMask(low_lying.And(flat_area))
    
    # Remove previous layers **safely**
    layer_names = [
        "Flood Frequency",
        "Flood-Prone Areas",
        "SRTM DEM",
        "Slope"
    ]
    for layer_name in layer_names:
        try:
            Map.remove_layer(layer_name)
        except Exception as e:
            pass  # Ignore errors if the layer is not found
    
    # Set the selected base map
    Map.add_basemap(selected_basemap)
    Map.centerObject(roi, 8)
    
    # Visualization parameters
    dem_vis = {"min": 0, "max": 50, "palette": ["ffffff", "00ff00", "007f00"]}
    slope_vis = {"min": 0, "max": 30, "palette": ["ffffff", "ffcc99", "ff3300"]}
    flood_prone_area_vis = {"min": 0, "max": 50, "palette": ["ffffff", "ff9999", "ff0000"]}
    flood_frequency_vis = {"min": 0, "max": 50, "palette": ["ffffff", "fffcb8", "0905ff"]}

    # Add layers again
    Map.addLayer(srtm.clip(roi), dem_vis, "SRTM DEM")
    Map.addLayer(slope.clip(roi), slope_vis, "Slope")
    Map.addLayer(flood_frequency.clip(roi), flood_frequency_vis, "Flood Frequency")
    Map.addLayer(flood_prone_area.clip(roi), flood_prone_area_vis, "Flood-Prone Areas")


# Set the button action
run_button.on_click(update_map)

# Display the widgets
display(city_selector, start_date_picker, end_date_picker, low_lying_input, base_map_selector, run_button)

# Ensure the base map is added initially
# Map.add_basemap("SATELLITE")  
Map.addLayerControl()
Map

Dropdown(description='City:', options=('Cuyuni/mazaruni (region N°7)', 'Demerara Mahaica (region N°4)', 'East …

DatePicker(value=datetime.date(1984, 3, 16), description='Start Date', step=1)

DatePicker(value=datetime.date(2024, 12, 31), description='End Date', step=1)

BoundedFloatText(value=10.0, description='Low-Lying Level (m)', step=1.0)

Dropdown(description='Base Map:', options=('SATELLITE', 'OSM'), value='SATELLITE')

Button(button_style='success', description='Run Analysis', style=ButtonStyle())

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [2]:
import ee
import geemap
import datetime
import ipywidgets as widgets
from IPython.display import display
import re  # Import regex module

# Initialize Earth Engine API
ee.Initialize()

# Create map
Map = geemap.Map()

# Load GAUL dataset and filter for Guyana
gaul_level1 = ee.FeatureCollection('FAO/GAUL/2015/level1')
guyana = gaul_level1.filter(ee.Filter.eq('ADM0_NAME', 'Guyana'))

# Get list of available cities in Guyana
raw_city_list = guyana.aggregate_array('ADM1_NAME').getInfo()

# Clean region names by removing "(region N°X)"
city_list = [re.sub(r"\s*\(region N°\d+\)", "", name) for name in raw_city_list]

# Define dynamic time period
start_date = datetime.date(1984, 3, 16)
end_date = datetime.date(2024, 12, 31)

# Convert to ee.Date objects
start_date_ee = ee.Date.fromYMD(start_date.year, start_date.month, start_date.day)
end_date_ee = ee.Date.fromYMD(end_date.year, end_date.month, end_date.day)

# Load datasets
jrc = ee.ImageCollection("JRC/GSW1_4/MonthlyHistory")
srtm = ee.Image("USGS/SRTMGL1_003")

# Interactive widgets
city_selector = widgets.Dropdown(
    options=city_list,
    value=city_list[0],
    description="City:",
)
start_date_picker = widgets.DatePicker(description="Start Date", value=start_date)
end_date_picker = widgets.DatePicker(description="End Date", value=end_date)
low_lying_input = widgets.BoundedFloatText(description="Low-Lying Level (m)", value=10, min=0, max=100, step=1)
base_map_selector = widgets.Dropdown(
    options=["SATELLITE", "OSM"],
    value="SATELLITE",
    description="Base Map:",
)
run_button = widgets.Button(description="Run Analysis", button_style="success")

    
def update_map(change):
    selected_city = city_selector.value
    
    # Match the cleaned city name with the original dataset
    raw_city_name = next(name for name in raw_city_list if selected_city in name)
    roi = guyana.filter(ee.Filter.eq('ADM1_NAME', raw_city_name)).geometry()
    
    start_date = start_date_picker.value
    end_date = end_date_picker.value
    low_lying_threshold = low_lying_input.value  # Get updated low-lying level
    selected_basemap = base_map_selector.value  # Get selected base map
    
    # Convert to ee.Date objects
    start_date_ee = ee.Date.fromYMD(start_date.year, start_date.month, start_date.day)
    end_date_ee = ee.Date.fromYMD(end_date.year, end_date.month, end_date.day)
    
    # Filter datasets to ROI
    jrc_filtered = jrc.filterBounds(roi).filterDate(start_date_ee, end_date_ee)
    permanent_water = ee.Image("JRC/GSW1_4/GlobalSurfaceWater").select("occurrence").gte(90)
    jrc_filtered = jrc_filtered.map(lambda img: img.updateMask(permanent_water.Not()))
    
    # Calculate flood frequency
    def calculate_flood_frequency(collection):
        def add_bands(img):
            obs = img.gt(0).rename("obs")
            water = img.select("water").eq(2).rename("water")
            return img.addBands([obs, water])
        
        collection = collection.map(add_bands)
        total_obs = collection.select("obs").sum().rename("total_obs")
        total_water = collection.select("water").sum().rename("total_water")
        flood_frequency = total_water.divide(total_obs).multiply(100).rename("flood_frequency")
        return flood_frequency.updateMask(flood_frequency.neq(0))
    
    flood_frequency = calculate_flood_frequency(jrc_filtered)
    slope = ee.Terrain.slope(srtm).rename("slope")
    flat_area = slope.lt(5).rename("flat_area")
    low_lying = srtm.lt(low_lying_threshold).rename("low_lying")
    flood_prone_area = flood_frequency.updateMask(low_lying.And(flat_area))
    
    # Remove previous layers **safely**
    layer_names = [
        "Flood Frequency",
        "Flood-Prone Areas",
        "SRTM DEM",
        "Slope"
    ]
    for layer_name in layer_names:
        try:
            Map.remove_layer(layer_name)
        except Exception:
            pass  # Ignore errors if the layer is not found
    
    # Set the selected base map
    Map.add_basemap(selected_basemap)
    Map.centerObject(roi, 8)
    
    # Visualization parameters
    dem_vis = {"min": 0, "max": 50, "palette": ["ffffff", "00ff00", "007f00"]}
    slope_vis = {"min": 0, "max": 30, "palette": ["ffffff", "ffcc99", "ff3300"]}
    flood_prone_area_vis = {"min": 0, "max": 50, "palette": ["ffffff", "ff9999", "ff0000"]}
    flood_frequency_vis = {"min": 0, "max": 50, "palette": ["ffffff", "fffcb8", "0905ff"]}

    # Add layers again
    Map.addLayer(srtm.clip(roi), dem_vis, "SRTM DEM")
    Map.addLayer(slope.clip(roi), slope_vis, "Slope")
    Map.addLayer(flood_frequency.clip(roi), flood_frequency_vis, "Flood Frequency")
    Map.addLayer(flood_prone_area.clip(roi), flood_prone_area_vis, "Flood-Prone Areas")


# Set the button action
run_button.on_click(update_map)

# Display the widgets
display(city_selector, start_date_picker, end_date_picker, low_lying_input, base_map_selector, run_button)

# Ensure the base map is added initially
# Map.add_basemap("SATELLITE")  
Map.addLayerControl()
Map

Dropdown(description='City:', options=('Barima Waini', 'Cuyuni/mazaruni', 'Demerara Mahaica', 'East Berbice/co…

DatePicker(value=datetime.date(1984, 3, 16), description='Start Date', step=1)

DatePicker(value=datetime.date(2024, 12, 31), description='End Date', step=1)

BoundedFloatText(value=10.0, description='Low-Lying Level (m)', step=1.0)

Dropdown(description='Base Map:', options=('SATELLITE', 'OSM'), value='SATELLITE')

Button(button_style='success', description='Run Analysis', style=ButtonStyle())

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

Basemap can only be one of the following:
  OpenStreetMap
  Esri.WorldStreetMap
  Esri.WorldImagery
  Esri.WorldTopoMap
  FWS NWI Wetlands
  FWS NWI Wetlands Raster
  NLCD 2021 CONUS Land Cover
  NLCD 2019 CONUS Land Cover
  NLCD 2016 CONUS Land Cover
  NLCD 2013 CONUS Land Cover
  NLCD 2011 CONUS Land Cover
  NLCD 2008 CONUS Land Cover
  NLCD 2006 CONUS Land Cover
  NLCD 2004 CONUS Land Cover
  NLCD 2001 CONUS Land Cover
  USGS NAIP Imagery
  USGS NAIP Imagery False Color
  USGS NAIP Imagery NDVI
  USGS Hydrography
  USGS 3DEP Elevation
  ESA Worldcover 2020
  ESA Worldcover 2020 S2 FCC
  ESA Worldcover 2020 S2 TCC
  ESA Worldcover 2021
  ESA Worldcover 2021 S2 FCC
  ESA Worldcover 2021 S2 TCC
  BaseMapDE.Color
  BaseMapDE.Grey
  BasemapAT.basemap
  BasemapAT.grau
  BasemapAT.highdpi
  BasemapAT.orthofoto
  BasemapAT.overlay
  BasemapAT.surface
  BasemapAT.terrain
  CartoDB.DarkMatter
  CartoDB.DarkMatterNoLabels
  CartoDB.DarkMatterOnlyLabels
  CartoDB.Positron
  CartoDB.PositronNoLa

# version 3 
* add open building

In [4]:
import ee
import geemap
import datetime
import ipywidgets as widgets
from IPython.display import display

# Initialize Earth Engine API
ee.Initialize()

# Create map
Map = geemap.Map()

# Load GAUL dataset and filter for Guyana
gaul_level1 = ee.FeatureCollection("FAO/GAUL/2015/level1")
guyana = gaul_level1.filter(ee.Filter.eq("ADM0_NAME", "Guyana"))

# Clean city names (remove "(region N°X)")
city_list = [name.split(" (")[0] for name in guyana.aggregate_array("ADM1_NAME").getInfo()]

# Define dynamic time period
start_date = datetime.date(1984, 3, 16)
end_date = datetime.date(2024, 12, 31)

# Convert to ee.Date objects
start_date_ee = ee.Date.fromYMD(start_date.year, start_date.month, start_date.day)
end_date_ee = ee.Date.fromYMD(end_date.year, end_date.month, end_date.day)

# Load datasets
jrc = ee.ImageCollection("JRC/GSW1_4/MonthlyHistory")
srtm = ee.Image("USGS/SRTMGL1_003")
buildings = ee.FeatureCollection("TIGER/2018/Buildings")  # Open building dataset

# Interactive widgets
city_selector = widgets.Dropdown(options=city_list, value=city_list[0], description="City:")
start_date_picker = widgets.DatePicker(description="Start Date", value=start_date)
end_date_picker = widgets.DatePicker(description="End Date", value=end_date)
low_lying_input = widgets.BoundedFloatText(description="Low-Lying Level (m)", value=10, min=0, max=100, step=1)
base_map_selector = widgets.Dropdown(options=["SATELLITE", "OSM"], value="SATELLITE", description="Base Map:")
run_button = widgets.Button(description="Run Analysis", button_style="success")


def update_map(change):
    selected_city = city_selector.value
    roi = guyana.filter(ee.Filter.eq("ADM1_NAME", selected_city)).geometry()
    
    start_date = start_date_picker.value
    end_date = end_date_picker.value
    low_lying_threshold = low_lying_input.value
    selected_basemap = base_map_selector.value
    
    # Convert to ee.Date objects
    start_date_ee = ee.Date.fromYMD(start_date.year, start_date.month, start_date.day)
    end_date_ee = ee.Date.fromYMD(end_date.year, end_date.month, end_date.day)
    
    # Filter datasets to ROI
    jrc_filtered = jrc.filterBounds(roi).filterDate(start_date_ee, end_date_ee)
    permanent_water = ee.Image("JRC/GSW1_4/GlobalSurfaceWater").select("occurrence").gte(90)
    jrc_filtered = jrc_filtered.map(lambda img: img.updateMask(permanent_water.Not()))
    
    # Calculate flood frequency
    def calculate_flood_frequency(collection):
        def add_bands(img):
            obs = img.gt(0).rename("obs")
            water = img.select("water").eq(2).rename("water")
            return img.addBands([obs, water])
        
        collection = collection.map(add_bands)
        total_obs = collection.select("obs").sum().rename("total_obs")
        total_water = collection.select("water").sum().rename("total_water")
        flood_frequency = total_water.divide(total_obs).multiply(100).rename("flood_frequency")
        return flood_frequency.updateMask(flood_frequency.neq(0))
    
    flood_frequency = calculate_flood_frequency(jrc_filtered)
    slope = ee.Terrain.slope(srtm).rename("slope")
    flat_area = slope.lt(5).rename("flat_area")
    low_lying = srtm.lt(low_lying_threshold).rename("low_lying")
    flood_prone_area = flood_frequency.updateMask(low_lying.And(flat_area))
    
    # Filter buildings in the selected region
    buildings_filtered = buildings.filterBounds(roi)

    # Identify flooded buildings
    flooded_buildings = buildings_filtered.filterBounds(flood_prone_area)

    # Count buildings
    total_buildings = buildings_filtered.size().getInfo()
    flooded_count = flooded_buildings.size().getInfo()

    print(f"Total Buildings in {selected_city}: {total_buildings}")
    print(f"Flooded Buildings: {flooded_count}")
    
    # Remove previous layers **safely**
    layer_names = [
        "Flood Frequency",
        "Flood-Prone Areas",
        "SRTM DEM",
        "Slope",
        "Buildings",
        "Flooded Buildings"
    ]
    for layer_name in layer_names:
        try:
            Map.remove_layer(layer_name)
        except Exception:
            pass  # Ignore errors if the layer is not found
    
    # Set the selected base map
    Map.add_basemap(selected_basemap)
    Map.centerObject(roi, 8)
    
    # Visualization parameters
    dem_vis = {"min": 0, "max": 50, "palette": ["ffffff", "00ff00", "007f00"]}
    slope_vis = {"min": 0, "max": 30, "palette": ["ffffff", "ffcc99", "ff3300"]}
    flood_prone_area_vis = {"min": 0, "max": 50, "palette": ["ffffff", "ff9999", "ff0000"]}
    flood_frequency_vis = {"min": 0, "max": 50, "palette": ["ffffff", "fffcb8", "0905ff"]}
    building_vis = {"color": "blue"}
    flooded_building_vis = {"color": "red"}

    # Add layers again
    Map.addLayer(srtm.clip(roi), dem_vis, "SRTM DEM")
    Map.addLayer(slope.clip(roi), slope_vis, "Slope")
    Map.addLayer(flood_frequency.clip(roi), flood_frequency_vis, "Flood Frequency")
    Map.addLayer(flood_prone_area.clip(roi), flood_prone_area_vis, "Flood-Prone Areas")
    Map.addLayer(buildings_filtered, building_vis, "Buildings")
    Map.addLayer(flooded_buildings, flooded_building_vis, "Flooded Buildings")


# Set the button action
run_button.on_click(update_map)

# Display the widgets
display(city_selector, start_date_picker, end_date_picker, low_lying_input, base_map_selector, run_button)

# Ensure the base map is added initially
Map.addLayerControl()
Map


Dropdown(description='City:', options=('Barima Waini', 'Cuyuni/mazaruni', 'Demerara Mahaica', 'East Berbice/co…

DatePicker(value=datetime.date(1984, 3, 16), description='Start Date', step=1)

DatePicker(value=datetime.date(2024, 12, 31), description='End Date', step=1)

BoundedFloatText(value=10.0, description='Low-Lying Level (m)', step=1.0)

Dropdown(description='Base Map:', options=('SATELLITE', 'OSM'), value='SATELLITE')

Button(button_style='success', description='Run Analysis', style=ButtonStyle())

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [7]:
import ee
import geemap
import datetime
import ipywidgets as widgets
from IPython.display import display

# Initialize Earth Engine API
ee.Initialize()

# Create map
Map = geemap.Map()

# Load GAUL dataset and filter for Guyana
gaul_level1 = ee.FeatureCollection("FAO/GAUL/2015/level2")
guyana = gaul_level1.filter(ee.Filter.eq("ADM0_NAME", "Guyana"))

# Clean city names (remove "(region N°X)")
city_list = [name.split(" (")[0] for name in guyana.aggregate_array("ADM1_NAME").getInfo()]

# Define dynamic time period
start_date = datetime.date(1984, 3, 16)
end_date = datetime.date(2024, 12, 31)

# Convert to ee.Date objects
start_date_ee = ee.Date.fromYMD(start_date.year, start_date.month, start_date.day)
end_date_ee = ee.Date.fromYMD(end_date.year, end_date.month, end_date.day)

# Load datasets
jrc = ee.ImageCollection("JRC/GSW1_4/MonthlyHistory")
srtm = ee.Image("USGS/SRTMGL1_003")
open_buildings = ee.FeatureCollection("GOOGLE/Research/open-buildings/v3/polygons")  # Google Open Buildings

# Interactive widgets
city_selector = widgets.Dropdown(options=city_list, value=city_list[0], description="City:")
start_date_picker = widgets.DatePicker(description="Start Date", value=start_date)
end_date_picker = widgets.DatePicker(description="End Date", value=end_date)
low_lying_input = widgets.BoundedFloatText(description="Low-Lying Level (m)", value=10, min=0, max=100, step=1)
base_map_selector = widgets.Dropdown(options=["SATELLITE", "OSM"], value="SATELLITE", description="Base Map:")
run_button = widgets.Button(description="Run Analysis", button_style="success")


# def update_map(change):
#     selected_city = city_selector.value
#     roi = guyana.filter(ee.Filter.eq("ADM1_NAME", selected_city)).geometry()
    
#     start_date = start_date_picker.value
#     end_date = end_date_picker.value
#     low_lying_threshold = low_lying_input.value
#     selected_basemap = base_map_selector.value
    
#     # Convert to ee.Date objects
#     start_date_ee = ee.Date.fromYMD(start_date.year, start_date.month, start_date.day)
#     end_date_ee = ee.Date.fromYMD(end_date.year, end_date.month, end_date.day)
    
#     # Filter datasets to ROI
#     jrc_filtered = jrc.filterBounds(roi).filterDate(start_date_ee, end_date_ee)
#     permanent_water = ee.Image("JRC/GSW1_4/GlobalSurfaceWater").select("occurrence").gte(90)
#     jrc_filtered = jrc_filtered.map(lambda img: img.updateMask(permanent_water.Not()))
    
#     # Calculate flood frequency
#     def calculate_flood_frequency(collection):
#         def add_bands(img):
#             obs = img.gt(0).rename("obs")
#             water = img.select("water").eq(2).rename("water")
#             return img.addBands([obs, water])
        
#         collection = collection.map(add_bands)
#         total_obs = collection.select("obs").sum().rename("total_obs")
#         total_water = collection.select("water").sum().rename("total_water")
#         flood_frequency = total_water.divide(total_obs).multiply(100).rename("flood_frequency")
#         return flood_frequency.updateMask(flood_frequency.neq(0))
    
#     flood_frequency = calculate_flood_frequency(jrc_filtered)
#     slope = ee.Terrain.slope(srtm).rename("slope")
#     flat_area = slope.lt(5).rename("flat_area")
#     low_lying = srtm.lt(low_lying_threshold).rename("low_lying")
#     flood_prone_area = flood_frequency.updateMask(low_lying.And(flat_area))
    
#     # Ensure flood_prone_area is valid
#     flood_prone_area = flood_prone_area.updateMask(flood_prone_area)  # Remove empty pixels

#     # Check if flood_prone_area has valid bands
#     if flood_prone_area.bandNames().size().getInfo() > 0:
#         flooded_buildings = open_buildings_filtered.filterBounds(flood_prone_area.geometry())
#     else:
#         flooded_buildings = ee.FeatureCollection([])  # Empty collection if no flood-prone areas

#     # Count buildings safely
#     open_buildings_filtered = open_buildings.filterBounds(roi)
#     total_buildings = open_buildings_filtered.size().getInfo()
#     flooded_count = flooded_buildings.size().getInfo()

#     print(f"Total Open Buildings in {selected_city}: {total_buildings}")
#     print(f"Flooded Open Buildings: {flooded_count}")



#     # # Filter open buildings in the selected region
#     # open_buildings_filtered = open_buildings.filterBounds(roi)

#     # # Identify flooded buildings
#     # flooded_buildings = open_buildings_filtered.filterBounds(flood_prone_area)

#     # # Count buildings
#     # total_buildings = open_buildings_filtered.size().getInfo()
#     # flooded_count = flooded_buildings.size().getInfo()

#     # print(f"Total Open Buildings in {selected_city}: {total_buildings}")
#     # print(f"Flooded Open Buildings: {flooded_count}")
    
#     # Remove previous layers **safely**
#     layer_names = [
#         "Flood Frequency",
#         "Flood-Prone Areas",
#         "SRTM DEM",
#         "Slope",
#         "Open Buildings",
#         "Flooded Open Buildings"
#     ]
#     for layer_name in layer_names:
#         try:
#             Map.remove_layer(layer_name)
#         except Exception:
#             pass  # Ignore errors if the layer is not found
    
#     # Set the selected base map
#     Map.add_basemap(selected_basemap)
#     Map.centerObject(roi, 8)
    
#     # Visualization parameters
#     dem_vis = {"min": 0, "max": 50, "palette": ["ffffff", "00ff00", "007f00"]}
#     slope_vis = {"min": 0, "max": 30, "palette": ["ffffff", "ffcc99", "ff3300"]}
#     flood_prone_area_vis = {"min": 0, "max": 50, "palette": ["ffffff", "ff9999", "ff0000"]}
#     flood_frequency_vis = {"min": 0, "max": 50, "palette": ["ffffff", "fffcb8", "0905ff"]}
#     open_building_vis = {"color": "blue"}
#     flooded_building_vis = {"color": "red"}

#     # Add layers again
#     Map.addLayer(srtm.clip(roi), dem_vis, "SRTM DEM")
#     Map.addLayer(slope.clip(roi), slope_vis, "Slope")
#     Map.addLayer(flood_frequency.clip(roi), flood_frequency_vis, "Flood Frequency")
#     Map.addLayer(flood_prone_area.clip(roi), flood_prone_area_vis, "Flood-Prone Areas")
#     Map.addLayer(open_buildings_filtered, open_building_vis, "Open Buildings")
#     Map.addLayer(flooded_buildings, flooded_building_vis, "Flooded Open Buildings")

# Load Open Buildings dataset
open_buildings = ee.FeatureCollection("GOOGLE/Research/open-buildings/v3/polygons")

# Function to filter open buildings by selected city
def get_open_buildings(region_geometry):
    return open_buildings.filterBounds(region_geometry)


def update_map(change):
    selected_city = city_selector.value
    roi = guyana.filter(ee.Filter.eq('ADM1_NAME', selected_city)).geometry()
    
    start_date = start_date_picker.value
    end_date = end_date_picker.value
    low_lying_threshold = low_lying_input.value
    selected_basemap = base_map_selector.value
    
    start_date_ee = ee.Date.fromYMD(start_date.year, start_date.month, start_date.day)
    end_date_ee = ee.Date.fromYMD(end_date.year, end_date.month, end_date.day)
    
    # Filter datasets to ROI
    jrc_filtered = jrc.filterBounds(roi).filterDate(start_date_ee, end_date_ee)
    permanent_water = ee.Image("JRC/GSW1_4/GlobalSurfaceWater").select("occurrence").gte(90)
    jrc_filtered = jrc_filtered.map(lambda img: img.updateMask(permanent_water.Not()))
    
    # Calculate flood frequency
    def calculate_flood_frequency(collection):
        def add_bands(img):
            obs = img.gt(0).rename("obs")
            water = img.select("water").eq(2).rename("water")
            return img.addBands([obs, water])
        
        collection = collection.map(add_bands)
        total_obs = collection.select("obs").sum().rename("total_obs")
        total_water = collection.select("water").sum().rename("total_water")
        flood_frequency = total_water.divide(total_obs).multiply(100).rename("flood_frequency")
        return flood_frequency.updateMask(flood_frequency.neq(0))
    
    flood_frequency = calculate_flood_frequency(jrc_filtered)
    slope = ee.Terrain.slope(srtm).rename("slope")
    flat_area = slope.lt(5).rename("flat_area")
    low_lying = srtm.lt(low_lying_threshold).rename("low_lying")
    flood_prone_area = flood_frequency.updateMask(low_lying.And(flat_area))
    
    # Load and filter open buildings
    open_buildings_filtered = get_open_buildings(roi)
    
    # Debug: Check if flood_prone_area has any bands
    flood_bands = flood_prone_area.bandNames().getInfo()
    print("Flood Prone Area Bands:", flood_bands)
    
    if flood_bands:
        flooded_buildings = open_buildings_filtered.filterBounds(flood_prone_area.geometry())
    else:
        print("No flood-prone areas detected.")
        flooded_buildings = ee.FeatureCollection([])  # Return empty collection
    
    # Count buildings safely
    total_buildings = open_buildings_filtered.size().getInfo()
    flooded_count = flooded_buildings.size().getInfo()
    
    print(f"Total Open Buildings in {selected_city}: {total_buildings}")
    print(f"Flooded Open Buildings: {flooded_count}")

    # Remove previous layers safely
    layer_names = ["Flood Frequency", "Flood-Prone Areas", "SRTM DEM", "Slope", "Open Buildings"]
    for layer_name in layer_names:
        try:
            Map.remove_layer(layer_name)
        except Exception as e:
            pass
    
    # Set the selected base map
    Map.add_basemap(selected_basemap)
    Map.centerObject(roi, 8)
    
    # Visualization parameters
    dem_vis = {"min": 0, "max": 50, "palette": ["ffffff", "00ff00", "007f00"]}
    slope_vis = {"min": 0, "max": 30, "palette": ["ffffff", "ffcc99", "ff3300"]}
    flood_prone_area_vis = {"min": 0, "max": 50, "palette": ["ffffff", "ff9999", "ff0000"]}
    flood_frequency_vis = {"min": 0, "max": 50, "palette": ["ffffff", "fffcb8", "0905ff"]}

    # Add layers again
    Map.addLayer(srtm.clip(roi), dem_vis, "SRTM DEM")
    Map.addLayer(slope.clip(roi), slope_vis, "Slope")
    Map.addLayer(flood_frequency.clip(roi), flood_frequency_vis, "Flood Frequency")
    if flood_bands:  # Only add if valid
        Map.addLayer(flood_prone_area.clip(roi), flood_prone_area_vis, "Flood-Prone Areas")
    
    # Add open buildings layer
    Map.addLayer(open_buildings_filtered, {"color": "blue"}, "Open Buildings")
    if flood_bands:
        Map.addLayer(flooded_buildings, {"color": "red"}, "Flooded Buildings")


# Set the button action
run_button.on_click(update_map)

# Display the widgets
display(city_selector, start_date_picker, end_date_picker, low_lying_input, base_map_selector, run_button)

# Ensure the base map is added initially
Map.addLayerControl()
Map


Dropdown(description='City:', options=('Cuyuni/mazaruni', 'Demerara Mahaica', 'East Berbice/corentyne', 'East …

DatePicker(value=datetime.date(1984, 3, 16), description='Start Date', step=1)

DatePicker(value=datetime.date(2024, 12, 31), description='End Date', step=1)

BoundedFloatText(value=10.0, description='Low-Lying Level (m)', step=1.0)

Dropdown(description='Base Map:', options=('SATELLITE', 'OSM'), value='SATELLITE')

Button(button_style='success', description='Run Analysis', style=ButtonStyle())

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

EEException: Image.multiply: If one image has no bands, the other must also have no bands. Got 0 and 1.

In [8]:
# Debug: Check if flood_prone_area is valid
print("Flood Prone Area Object:", flood_prone_area.getInfo())  


NameError: name 'flood_prone_area' is not defined